<a id='0.1'></a>
## 0.1 load required packages

In [1]:
%run "..\..\..\Startup_py3.py"
sys.path.append(r"..\..\..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

# library design specific tools
from ImageAnalysis3.library_tools import LibraryDesigner as ld
from ImageAnalysis3.library_tools import LibraryTools as lt
# biopython imports
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

29792


In [2]:
## Some folders
# human genome
reference_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl'
genome_folder = os.path.join(reference_folder, 'Genome')
# Library directories
pool_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-10_Aire'

In [4]:
resolution = 0
flanking = 10000
# folder for sub-pool
library_folder = os.path.join(pool_folder, f'Genes_TSS_DNA')
if not os.path.exists(library_folder):
    print(f"create library folder: {library_folder}")
    os.makedirs(library_folder)
# folder for fasta sequences
sequence_folder = os.path.join(library_folder, 'sequences')
if not os.path.exists(sequence_folder):
    print(f"create sequence folder: {sequence_folder}")
    os.makedirs(sequence_folder)
# folder to save result probes
report_folder = os.path.join(library_folder, 'reports')
if not os.path.exists(report_folder):
    print(f"create report folder: {report_folder}")
    os.makedirs(report_folder)
    
print(f"-- library_folder: {library_folder}")
print(f"-- sequence_folder: {sequence_folder}")
print(f"-- report_folder: {report_folder}")

-- library_folder: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-10_Aire\Genes_TSS_DNA
-- sequence_folder: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences
-- report_folder: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-10_Aire\Genes_TSS_DNA\reports


## 1.1 load gene list

In [3]:
gene_list_folder = os.path.join(pool_folder, 'Gene_list')
gene_list_filename = os.path.join(gene_list_folder, 'uniqued_clustered_genes_for_yuan_2021-04-22.txt')
import pandas as pd
gene_df = pd.read_csv(gene_list_filename, delimiter = "\t", header=None)
gene_df.columns = ['Cluster', 'Gene']
gene_df

,Cluster,Gene
0,Immature MEC,Ccl21a
1,Immature MEC,Krt14
2,Immature MEC,Krt5
3,Immature MEC,Col6a1
4,Immature MEC,Lifr
...,...,...
204,Aire-stage,Ltf
205,Aire-stage,Clps
206,Aire-stage,Col1a1
207,Aire-stage,Gpx6


In [55]:
np.unique(gene_df['Gene'])

array(['Acpp', 'Agt', 'Aldh3a1', 'Aldob', 'Alox5ap', 'Alpi', 'Alpl',
       'Ang4', 'Anxa8', 'Apoa1', 'Apoa2', 'Apoc2', 'Apod', 'Ascl1',
       'Avil', 'Bgn', 'C1qb', 'C3', 'Calb2', 'Cald1', 'Camp', 'Car4',
       'Car8', 'Ccdc30', 'Cck', 'Ccl1', 'Ccl2', 'Ccl20', 'Ccl21a', 'Ccl6',
       'Ccl9', 'Cd14', 'Cebpb', 'Celf3', 'Cga', 'Cilp', 'Ckb', 'Ckm',
       'Clps', 'Cnfn', 'Col1a1', 'Col4a1', 'Col6a1', 'Cpn1', 'Crabp1',
       'Cryaa', 'Ctrb1', 'Cxcl13', 'Cyp1a2', 'Cyp2f2', 'Dbn1', 'Dclk1',
       'Dlk1', 'Dppa3', 'Ecm1', 'Efna3', 'Elovl4', 'Fabp1', 'Fabp6',
       'Fam183b', 'Fam25c', 'Fgg', 'Foxa1', 'Foxa2', 'Foxa3', 'Foxj1',
       'Fxyd3', 'Fxyd6', 'Gabrp', 'Gal', 'Gatm', 'Ghrl', 'Gm13539',
       'Gm2a', 'Gm94', 'Gp2', 'Gpx3', 'Gpx6', 'Grhl1', 'Gsdma', 'Gsta2',
       'Gsta4', 'Gstm3', 'Gsto1', 'Guca2a', 'H19', 'Hamp2', 'Hba-a2',
       'Hbb-y', 'Hck', 'Hnf4a', 'Hnf4g', 'Hp', 'Iapp', 'Idi2', 'Ifi27l2b',
       'Igf1', 'Igf2', 'Igfbp5', 'Igfbp6', 'Ins2', 'Isg15', 'Itga6',
       'Jp

In [5]:
# laod encoding
# summarize total readout usage
encoding_folder = os.path.join(pool_folder, f'Genes_intronic_RNA')
gene_2_readout_dict = pickle.load(open(os.path.join(encoding_folder, 'gene_2_readout.pkl'), 'rb'))

In [24]:
gene_2_readout_dict

{'Ccl21a': ['u0', 'c7', 'c26', 'c35'],
 'Krt14': ['u1', 'c23', 'c34', 'c47'],
 'Krt5': ['u2', 'c13', 'c17', 'c34'],
 'Col6a1': ['u3', 'c2', 'c23', 'c43'],
 'Lifr': ['u4', 'c2', 'c26', 'c38'],
 'Itga6': ['u5', 'c16', 'c22', 'c49'],
 'Col4a1': ['u6', 'c9', 'c10', 'c41'],
 'Ascl1': ['u7', 'c26', 'c29', 'c41'],
 'Cald1': ['u8', 'c10', 'c23', 'c39'],
 'Krt17': ['u9', 'c7', 'c48', 'c49'],
 'C3': ['u10', 'c9', 'c20', 'c27'],
 'Trp63': ['u11', 'c17', 'c32', 'c40'],
 'Dclk1': ['u12', 'c1', 'c6', 'c13'],
 'Trpm5': ['u13', 'c6', 'c37', 'c43'],
 'Ly6g6f': ['u14', 'c3', 'c12', 'c31'],
 'Avil': ['u15', 'c11', 'c15', 'c49'],
 'Alox5ap': ['u16', 'c9', 'c38', 'c44'],
 'Hck': ['u17', 'c0', 'c17', 'c28'],
 'Plk2': ['u18', 'c5', 'c13', 'c22'],
 'Pik3r5': ['u19', 'c27', 'c46', 'c47'],
 'Agt': ['u20', 'c6', 'c29', 'c34'],
 'Nrgn': ['u21', 'c5', 'c25', 'c42'],
 'Plac8': ['u22', 'c7', 'c12', 'c20'],
 'Fxyd6': ['u23', 'c11', 'c34', 'c46'],
 'Pou2f3': ['u24', 'c33', 'c40', 'c47'],
 'Ccl20': ['u25', 'c3', 'c13',

In [7]:
# load used readouts
readout_usage_file = os.path.join(library_folder, 'readout_usage.pkl')
readout_dict = pickle.load(open(readout_usage_file, 'rb'))

In [8]:
readout_dict

{'c': [SeqRecord(seq=Seq('AACCACGGGCTAGAAATATCCGAGTGGCAT'), id='NDB_784', name='NDB_784', description='NDB_784', dbxrefs=[]),
  SeqRecord(seq=Seq('GCCTAAGACGTGCGACATGTTATGCTAGGC'), id='NDB_755', name='NDB_755', description='NDB_755', dbxrefs=[]),
  SeqRecord(seq=Seq('GCCGTTTCGTATATGTCGATGTCCTTAGAC'), id='NDB_826', name='NDB_826', description='NDB_826', dbxrefs=[]),
  SeqRecord(seq=Seq('TACAAATACCGGTGGTTGTCACTTTCCGAA'), id='NDB_713', name='NDB_713', description='NDB_713', dbxrefs=[]),
  SeqRecord(seq=Seq('TCCGTGACGAGATCCCGGTCGGATATGTGA'), id='NDB_865', name='NDB_865', description='NDB_865', dbxrefs=[]),
  SeqRecord(seq=Seq('GAGCTTATGACTACCACGGCATCGATTTAG'), id='NDB_725', name='NDB_725', description='NDB_725', dbxrefs=[]),
  SeqRecord(seq=Seq('CGCGAAGATCGAACTCGTTGACTAAGCATT'), id='NDB_817', name='NDB_817', description='NDB_817', dbxrefs=[]),
  SeqRecord(seq=Seq('GTGGTAAACCTCAACTACTAAGCCGCGGTA'), id='NDB_710', name='NDB_710', description='NDB_710', dbxrefs=[]),
  SeqRecord(seq=Seq('CCAGTA

In [25]:
# generate the codebook
codebook = pd.DataFrame(columns=['name','id']+[_r.id for _r in readout_dict['c']])
#codebook.add(['name', 'id'], axis=1)
codebook['name'] = list(gene_2_readout_dict.keys())

In [29]:
codebook.loc[codebook['name']=='Ccl21a']

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_715,NDB_698,NDB_883,NDB_863,NDB_1033,NDB_878,NDB_631,NDB_758,NDB_1075,NDB_653
0,Ccl21a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Index(['NDB_784', 'NDB_755', 'NDB_826', 'NDB_713', 'NDB_865', 'NDB_725',
       'NDB_817', 'NDB_710', 'NDB_652', 'NDB_971', 'NDB_718', 'NDB_743',
       'NDB_847', 'NDB_683', 'NDB_643', 'NDB_815', 'NDB_760', 'NDB_965',
       'NDB_790', 'NDB_917', 'NDB_778', 'NDB_728', 'NDB_844', 'NDB_974',
       'NDB_961', 'NDB_746', 'NDB_661', 'NDB_875', 'NDB_901', 'NDB_773',
       'NDB_868', 'NDB_812', 'NDB_1027', 'NDB_716', 'NDB_754', 'NDB_860',
       'NDB_856', 'NDB_680', 'NDB_634', 'NDB_737', 'NDB_715', 'NDB_698',
       'NDB_883', 'NDB_863', 'NDB_1033', 'NDB_878', 'NDB_631', 'NDB_758',
       'NDB_1075', 'NDB_653'],
      dtype='object')

In [36]:
for _gname, _bits in gene_2_readout_dict.items():
    binary_code = []
    for _i in range(50):
        if f"c{_i}" in _bits:
            binary_code.append(1)
        else:
            binary_code.append(0)
    
    codebook.loc[codebook['name']==_gname, codebook.columns[2:]] = binary_code
    codebook.loc[codebook['name']==_gname,'id'] = int(_bits[0].split('u')[1])

In [41]:
chr_2_region_num = pickle.load(open(os.path.join(r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-10_Aire\Genes_intronic_RNA',
                                                 'chr_2_region_num.pkl'), 'rb'))

In [42]:
chr_2_region_num

{'4': 14,
 '11': 30,
 '15': 14,
 '10': 7,
 '2': 14,
 '8': 8,
 '6': 10,
 '17': 10,
 '16': 8,
 '3': 13,
 '7': 21,
 '5': 7,
 '13': 4,
 '9': 16,
 '1': 9,
 '14': 9,
 '12': 8,
 '18': 2,
 '19': 4,
 'X': 1}

In [43]:
chr_2_gene_names = pickle.load(open(os.path.join(r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-10_Aire\Genes_intronic_RNA',
                                                 'chr_2_gene_names.pkl'), 'rb'))

In [44]:
chr_2_gene_names

{'4': ['Penk',
  'Car8',
  'Cga',
  'Ccl21a',
  'Spag8',
  'Aldob',
  'Pdzk1ip1',
  'Tspan1',
  'Ccdc30',
  'Guca2a',
  'C1qb',
  'Alpl',
  'Nppa',
  'Isg15'],
 '11': ['Nefh',
  'Hba-a2',
  'Gabrp',
  'Fabp6',
  'Gpx3',
  'Gm2a',
  'Sparc',
  'Fam183b',
  'Aldh3a1',
  'Pik3r5',
  'Rap1gap2',
  'Ccl2',
  'Ccl1',
  'Ccl9',
  'Ccl6',
  'Car4',
  'Mpo',
  'Col1a1',
  'Gsdma',
  'Krt10',
  'Krt13',
  'Krt19',
  'Krt9',
  'Krt14',
  'Krt16',
  'Krt17',
  'Pyy',
  'Jpt1',
  'Foxj1',
  'Timp2'],
 '15': ['Lifr',
  'Tnfrsf11b',
  'Psca',
  'Lypd2',
  'Ly6d',
  'Ly6c1',
  'Ly6c2',
  'Lgals2',
  'Odf3b',
  'Krt5',
  'Krt71',
  'Krt77',
  'Krt76',
  'Igfbp6'],
 '10': ['Vpreb3', 'Col6a1', 'Ascl1', 'Igf1', 'Nts', 'Avil', 'Tac2'],
 '2': ['Meig1',
  'Ptgds',
  'Lcn3',
  'Gm13539',
  'Olfm1',
  'Lcn2',
  'Itga6',
  'Gatm',
  'Mal',
  'Foxa2',
  'Hck',
  'Hnf4a',
  'Slpi',
  'Cebpb'],
 '8': ['Col4a1', 'Plvap', 'Mt4', 'Mt3', 'Hp', 'Calb2', 'Ctrb1', 'Agt'],
 '6': ['Tac1',
  'Cald1',
  'Npy',
  'Tacstd2',
 

In [45]:
# generate gene_to_chr_id
gene_2_chr = {}
gene_2_chr_order = {}
for _chr, _genes in chr_2_gene_names.items():
    for _i, _gene in enumerate(_genes):
        gene_2_chr[_gene] = _chr
        gene_2_chr_order[_gene] = _i

In [47]:
codebook['chr'] = [gene_2_chr[_g] for _g in codebook['name']]
codebook['chr_order'] = [gene_2_chr_order[_g] for _g in codebook['name']]

In [48]:
codebook

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_883,NDB_863,NDB_1033,NDB_878,NDB_631,NDB_758,NDB_1075,NDB_653,chr,chr_order
0,Ccl21a,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,4,3
1,Krt14,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,11,23
2,Krt5,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,15,9
3,Col6a1,3,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,10,1
4,Lifr,4,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,Ltf,204,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9,14
205,Clps,205,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,17,1
206,Col1a1,206,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,11,17
207,Gpx6,207,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,13,1


In [49]:
codebook.to_csv(r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-10_Aire\Summary_tables\CTP10-Aire_codebook.csv',
                index=None)